<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sentiment analysis con Bag of Words

### Objetivo
El objetivo es utilizar las críticas de películas para que el sistema determine si la evaluación es positiva o negativa (sentiment analysis como clasificador binario de texto)

In [ ]:
import numpy as np
import random
import io
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def curva_roc(model, X_test, y_test):
    y_hat_prob = model.predict(X_test).ravel()

    # Calcular la exactitud (accuracy)
    scores = model.evaluate(X_test, y_test)
    print("Accuracy:", scores[1])

    fpr, tpr, thresholds = roc_curve(y_test, y_hat_prob)
    auc_keras = auc(fpr, tpr)
    print('auc_keras', auc_keras)

    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc_keras))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('Curva ROC test')
    plt.legend(loc='best')
    plt.show()

### Datos
Utilizaremos como dataset críticas de películas de IMDB puntuadas deforma positiva o negativa.\
Referencia del dataset: [LINK](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

In [ ]:
# Descargar la carpeta de dataset
!curl -L -o 'imdb_dataset.csv' 'https://drive.google.com/u/0/uc?id=1el8tS8JZhPLd2FQWPHpvG_AIyW5np3h9&export=download&confirm=t'

In [ ]:
# Armar el dataset
df = pd.read_csv('imdb_dataset.csv')
df.head()

### 1 - Limpieza de datos
- En los datos se observo que en la columna "review" hay código HTML de salto de línea.
- Tranformar la columna snetiment a 0 y 1



In [ ]:
# En los datos se observó código HTML de salto de línea <br />
import re
df_reviews = df.copy() 
df_reviews['review'] = df['review'].apply(lambda x: re.sub("<br />", "", x))
df_reviews['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values
df_reviews.head()

In [ ]:
# Observar como está distribuido el dataset respecto a la columna Rating
# es decir, observar que tan balanceado se encuentra respecto a cada clase
df_reviews['sentiment'].value_counts()

In [ ]:
# Observar como está distribuido el dataset
sns.countplot(x='sentiment', data=df_reviews)
plt.show()

Se puede observar que el dataset está perfectamente balanceado

In [ ]:
# Tomar la columna de las review y almacenarlo todo en un vector numpy de reviews
text_sequences = df_reviews['review'].values
text_sequences.shape

In [ ]:
# Cuantas reviews (rows) hay para evaluar?
len(text_sequences)

In [ ]:
# - Por defecto CountVectorizer elimina los signos de puntuacion y transforma
# todas las palabras a lowercase
# - max_features --> limita la máxima dimensión del oneHotEncoding (max vocab_size)
# - stop_words --> quitamos aquellas palabras que para el idioma no se consideran
# relevantes (como los artículos, pronombres, preposiciones, adverbios, etc)
# - Referencia:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=2000, stop_words='english')
X = vectorizer.fit_transform(text_sequences).toarray()

In [ ]:
# Los datos de entrada (X) son un vector de oneHotEncoding del tamaño
# del vocabulario y de la cantidad de filas
X.shape

In [ ]:
# Tomar la columna rating y alcemacenarla en una variable "y"
# Su shape debe ser equivalente la cantidad de rows del corpus
y = df_reviews['sentiment'].values
print(y.shape)

In [ ]:
# Dividir los datos en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Determinar la dimensiones de entrada y salida
in_shape = X_train.shape[1] # max input sentence len
out_shape = 1 # binary classification
print("in_shape", in_shape, ", out_shape", out_shape)

### 2 - Entrenar el modelo DNN con BOW

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Armar un modelo de clasificacion binaria con DNN
model = Sequential()
model.add(Dense(units=128, activation='relu', input_shape=(in_shape,)))
model.add(Dropout(rate=0.3))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=out_shape, activation='sigmoid'))

model.compile(optimizer="Adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
hist = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
epoch_count = range(1, len(hist.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=hist.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# Como este modelo es binario podemos calcular la curva ROC
curva_roc(model, X_test, y_test)

### 3 - Entrenar un modelo previamente reduciendo el vector de entrada (X)

In [ ]:
# Un vector de 2000 columnas es demasiado grande para entrenar un modelo clásico
# de deep learning (DNN)
# Se utiliza PCA para reducir la dimensionalidad
from sklearn.decomposition import PCA
X_pca = PCA(n_components=50).fit_transform(X)

In [ ]:
X_pca.shape

In [ ]:
# Dividir los datos en train y test
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [ ]:
# Determinar la dimensiones de entrada y salida
in_shape = X_train2.shape[1] # max input sentence len
out_shape = 1 # binary classification
print("in_shape", in_shape, ", out_shape", out_shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Utilizar la misma estructura de modelo del punto anterior
model2 = Sequential()
model2.add(Dense(units=128, activation='relu', input_shape=(in_shape,)))
model2.add(Dropout(rate=0.3))
model2.add(Dense(units=64, activation='relu'))
model2.add(Dropout(rate=0.5))
model2.add(Dense(units=32, activation='relu'))
model2.add(Dropout(rate=0.5))
model2.add(Dense(units=out_shape, activation='sigmoid'))

model2.compile(optimizer="Adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model2, to_file='model2_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
hist2 = model2.fit(X_train2, y_train2, epochs=20, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
epoch_count = range(1, len(hist2.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=hist2.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=hist2.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
model2.evaluate(X_test2, y_test2)

In [ ]:
# Como este modelo es binario podemos calcular la curva ROC
curva_roc(model2, X_test2, y_test2)

### 4 - Conclusión
El modelo con "bag of words" resultó ser muy fácil de armar, barato de entrenar (liviano) y obtuvo una muy buena performance.\
El modelo de entrada completa (oneHotEncoding) performó mejor que el dimensión reducida con PCA pero realizó mucho overfitting.